## Детектирование аномалий: лабораторная работа 
#### Имя Фамилия, группа

In [1]:
import wfdb
import matplotlib.pyplot as plt
import numpy as np
from librosa.sequence import dtw
import scipy.signal as signal
from sklearn.manifold import TSNE

In [2]:
annotation = wfdb.rdann('mit-bih-arrhythmia-database-1.0.0/100', 'atr').__dict__
data = wfdb.rdsamp('mit-bih-arrhythmia-database-1.0.0/100')[0][:,0]
sample = np.array(annotation['sample'])
labels = np.array(annotation['symbol'])

В этот раз у вас будет одно задание, но более длинное. Вам предлагается проанализировать кардиосигнал из датасета MIT BIH Arrhythmia. ( https://www.physionet.org/content/mitdb/1.0.0/ ). Данные представляют из себя длинный временной ряд с частотой дискретизации (sampling rate) 360 Гц. Кроме того, авторы датасета нашли пиковые значения сигнала, соответствующие зубцу "R" кардиограммы (подробнее о "зубцах" можете прочитать здесь: https://en.wikipedia.org/wiki/QRS_complex ). В sample хранится информация о координатах R-зубцов, в labels -- разметка, где "N" - означает нормальный сердечный ритм, а "A" - аномальный, вызванный сердечной аритмией. Посмотрим как это выглядит, на примере первых 10 секунд записи:

In [ ]:
start = 0
stop = 3600 #10 секунд
y = labels[sample < stop]
x = sample[sample < stop]

y = y[x > start]
x = x[x > start]

x2 = x[y != 'N']
x = x[y == 'N']

plt.figure(figsize=(20,5))
plt.plot(np.arange(start, stop), data[start:stop])
plt.scatter(x, data[x])
plt.scatter(x2, data[x2], alpha = 1)

Ваша задача - автоматически детектировать аномальные сердечные ритмы. Для этого вам нужно научиться считать попарные расстояния между сегментами сигнала с помощью алгоритма Dynamic Time Warping. Вам разрешается пользоваться координатами R-зубцов, но при анализе не разрешается пользоваться самой разметкой ('N' vs 'A').

Задание 4.1 (2 балла): EDA и предобработка
Визуализируйте данные для всех аномальных сердечных ритмов. Опишите, чем они отличаются от нормальных. Нуждаются ли данные в предобработке? Если да, то почему и в какой? Подсказка: https://www.youtube.com/watch?v=rRdmrY5e-N8 . Можете воспользоваться функцией remove_baseline_wandering или попробовать другие подходы. Визуализируйте результат.

In [ ]:
def remove_baseline_wandering(ecg_data, sampling_rate):
    nyquist = 0.5 * sampling_rate
    low = 0.5 / nyquist  # Low frequency cutoff (0.5 Hz)
    b, a = signal.butter(1, low, btype='high')
    filtered_ecg = signal.filtfilt(b, a, ecg_data)
    return filtered_ecg



#### Задание 4.2 (2 балла)
Разделите сигнал на сегменты, в качестве границ сегментов возьмите координаты, соответствующие детектированным "пиковым" значениям. Опробуйте метод DTW для выравнивания двух сегментов, когда (а) оба сегмента принадлежат классу 'N', (б) сегменты принадлежат разным классам. Визуализируйте результат (постройте на одних осях координат сигнал и выровненный на него другой сигнал). Приведите в явном виде "стоимость" выравнивания для пунктов (а) и (б). Можете воспользоваться готовой реализацией алгоритма DTW (например https://librosa.org/doc/main/generated/librosa.sequence.dtw.html ) или написать его сами (+1 балл). Предложите подходящее поточечное расстояние для оптимального нахождения спрямляющего пути (возможно, вы захотите по-разному сравнивать значения около нуля и около 1)

#### Задание 4.3 (2 балла).
Постройте матрицу попарных расстояний между сегментами. Без оптимизации выполнение этой задачи может занять около часа! Можете ли вы как-либо ускорить время выполнения, наложив какие-либо ограничения на алгоритм? (+1 балл)

#### Задание 4.4 (2 балла)
Пользуясь одним из методов детектирования аномалий, основанном на попарных расстояниях, оцените "аномальность" каждого сегмента. Интерпретируйте полученные результаты, используя ground-truth метки сегментов. Посчитайте качество детектирования аномалий.

#### Задание 4.5 (2 балла)

Используйте полученную ранее матрицу попарных расстояний в качестве инпута алгоритма t-SNE для визуализации данных (используйте plt.scatter). Отметьте на диаграмме аномальные элементы.